In [2]:
import os
import vllm
from hidden_states import get_all_hidden_states
from vllm import LLM

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.environ["VLLM_USE_V1"] = "0"

model_name = "/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/"
llm = LLM(model=model_name, task="reward", tensor_parallel_size=1)

INFO 07-10 16:10:43 [config.py:1472] Using max model len 32768
INFO 07-10 16:10:43 [config.py:4615] Only "last" pooling supports chunked prefill and prefix caching; disabling both.
INFO 07-10 16:10:43 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294.d20250710) with config: model='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.44it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.44it/s]


INFO 07-10 16:10:47 [default_loader.py:272] Loading weights took 0.83 seconds


INFO 07-10 16:10:47 [model_runner.py:1255] Model loading took 2.8876 GiB and 0.997674 seconds


In [3]:
slow_thinking=["Please give the detailed thinking progress.","Output the complete thinking steps."]
fast_thinking=["Output the answer directly.","Just give me the answer."]
texts = []
for t in slow_thinking+fast_thinking:
    texts.append(f"""<|im_start|>user
{t}<|im_end|>
<|im_start|>assistant
That""")
all_hidden_states, outputs = get_all_hidden_states(llm, texts)

Processed prompts: 100%|██████████| 4/4 [00:00<00:00,  4.27it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


In [5]:
from steer import (extract_statistical_control_vector,
    extract_diffmean_control_vector,
    extract_pca_control_vector,
    extract_lat_control_vector,
    extract_gemmascope_sae_diffmean_control_vector,
    extract_linear_probe_control_vector,
    StatisticalControlVector
)

In [7]:
control_vector = extract_diffmean_control_vector(
    all_hidden_states=all_hidden_states,  # 三维列表 [样本][layer][token]
    positive_indices=[0, 1],     # 正样本索引
    negative_indices=[2, 3],     # 负样本索引
    model_type="qwen2.5",
    token_pos=-1,      # 使用最后一个token（默认）
    normalize=True
)
control_vector.export_gguf("thinking_switch_control_vector.gguf")
control_vector = StatisticalControlVector.import_gguf("thinking_switch_control_vector.gguf")
print(control_vector)

Computing DiffMean directions: 100%|██████████| 28/28 [00:00<00:00, 9812.88it/s]


StatisticalControlVector(model_type='qwen2.5', method='diffmean', directions={0: memmap([-0.03187475,  0.01142179, -0.03160913, ..., -0.00478121,
         0.01248428,  0.00790228], shape=(1536,), dtype=float32), 1: memmap([ 0.00872863, -0.01371641,  0.00514366, ..., -0.03647319,
        -0.03070606, -0.00280563], shape=(1536,), dtype=float32), 2: memmap([ 0.02447614,  0.0163406 ,  0.00570183, ..., -0.03365469,
         0.00344196,  0.01446317], shape=(1536,), dtype=float32), 3: memmap([ 0.01258381,  0.01209226, -0.0098311 , ..., -0.03342575,
        -0.02204625, -0.00629191], shape=(1536,), dtype=float32), 4: memmap([-0.00981627,  0.02515419,  0.02270012, ..., -0.01525028,
        -0.00219113, -0.02646887], shape=(1536,), dtype=float32), 5: memmap([-0.00832835,  0.03097105, -0.00659328, ...,  0.00589925,
         0.01752423, -0.00416417], shape=(1536,), dtype=float32), 6: memmap([ 0.0026915 ,  0.01034547, -0.00302794, ...,  0.01000903,
        -0.00765397, -0.0211956 ], shape=(1536,), 